# Lexikalische Merkmale als Sparse Matrix

In den meisten computerlinguistischen Anwendungen des maschinellen Lernens spielen lexikalische Merkmale (wie z.B. die Bag-of-words-Repräsentation von Texten) eine wichtige Rolle. Dabei entsteht oft eine sehr große, aber dünn besetze Mermalsmatrix (engl. **Sparse Matrix**), die über eine Milliarde Werte enthalten kann, von denen jedoch nur ein kleiner Prozentsatz von null verschieden (_nonzero_) ist. Die Verarbeitung solcher Merkmalsmatrizen ist nur mit speziellen, effizienten Speicherformaten praktikabel, die vom **SciPy**-Paket zur Verfügung gestellt werden. Dort finden sich auch zusätzliche Algorithmen aus der linearen Algebra für NumPy-Matrizen.

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
import re

Als Beispieldaten ziehen wir ein kurzes, sehr [bekanntes Gedicht](https://www.lyrikline.org/de/gedichte/ottos-mops-1232) heran, dessen Zeilen wir durch lexikalische und textstatistische Merkmale beschreiben wollen.

In [2]:
ottos_mops = """
ottos mops trotzt
otto: fort mops fort
ottos mops hopst fort
otto: soso

otto holt koks
otto holt obst
otto horcht
otto: mops mops
otto hofft

ottos mops klopft
otto: komm mops komm
ottos mops kommt
ottos mops kotzt
otto: ogottogott
"""

Die Vorverarbeitung und Tokenisierung dieses Texts gestaltet sich sehr einfach, da alle Token durch Leerzeichen und gelegentlich einen Doppelpunkt begrenzt sind.

In [3]:
corpus = [re.split(r':?\s+', x) for x in ottos_mops.splitlines() if x != ""]
corpus

[['ottos', 'mops', 'trotzt'],
 ['otto', 'fort', 'mops', 'fort'],
 ['ottos', 'mops', 'hopst', 'fort'],
 ['otto', 'soso'],
 ['otto', 'holt', 'koks'],
 ['otto', 'holt', 'obst'],
 ['otto', 'horcht'],
 ['otto', 'mops', 'mops'],
 ['otto', 'hofft'],
 ['ottos', 'mops', 'klopft'],
 ['otto', 'komm', 'mops', 'komm'],
 ['ottos', 'mops', 'kommt'],
 ['ottos', 'mops', 'kotzt'],
 ['otto', 'ogottogott']]

## Bag-of-words als Sparse Matrix

Wir erstellen die Sparse-Matrix-Darstellung eines Bag-of-words (**BOW**) zunächst manuell, hier am Beispiel von drei ausgewählten Gedichtzeilen:

In [4]:
[corpus[i] for i in (7, 10, 11)]

[['otto', 'mops', 'mops'],
 ['otto', 'komm', 'mops', 'komm'],
 ['ottos', 'mops', 'kommt']]

Dazu müssen die Wort-Typen auf die Spalten der BOW-Matrix abbilden; die Zeilen der Matrix entsprechen einfach den drei Gedichtzeilen. Wir benötigen also ein Lexikon, das jedem Typ einen Spaltenindex als numerische ID zuweist, z.B.

In [5]:
lexicon = {'otto': 0, 'ottos': 1, 'mops': 2, 'komm': 3, 'kommt': 4}

Eine Sparse Matrix speichert nur diejenigen Einträge, die von null verschieden sind.  Üblicherweise handelt es sich dabei um nichtnegative Matrizen (was aber nicht notwendigerweise der Fall sein muss).  Zur Erstellung einer solchen Sparse Matrix müssen wir die Positionen aller von null verschiedenen Elemente sowie die zugehörigen Werte angeben.

Beispielsweise steht in der zweiten Zeile (`r = 1`) in Spalte `c = lexicon['komm']` der Wert 2. Hier erstellen wir die entsprechenden Listen von Hand. In der Übung lernen Sie, wie ein solcher BOW mit Python-Code erzeugt werden kann.

In [6]:
r = [0,      0,      1,      1,      1,      2,       2,      2]  # in echtem Code mit effizienterem np.array!
w = ['otto', 'mops', 'otto', 'mops', 'komm', 'ottos', 'mops', 'kommt']
c = [lexicon[x] for x in w]
f = [1,      2,      1,      1,      2,      1,       1,      1]
[r, w, c, f]

[[0, 0, 1, 1, 1, 2, 2, 2],
 ['otto', 'mops', 'otto', 'mops', 'komm', 'ottos', 'mops', 'kommt'],
 [0, 2, 0, 2, 3, 1, 2, 4],
 [1, 2, 1, 1, 2, 1, 1, 1]]

SciPy unterstützt verschiedene Sparse-Matrix-Formate. Wir wählen hier das _**coo**ordinate format_, das den von uns erstellten Tripeln enstpricht. Wir können mit `dtype` den Datentyp der gespeicherten Werte festlegen; ohne die Angabe würde aus `f` ein Integer-Format abgeleitet. (**NB:** SciPy stellt die Schnittstelle gerade auf Sparse Arrays um, die besser mit NumPy-Arrays kompatibel sind. Sparse Arrays sindaber noch nicht vollständig implementiert und nur in allerneusten SciPy-Versionen enthalten.)

In [7]:
M = sp.sparse.coo_matrix((f, (r, c)), dtype=np.float64)
M

<3x5 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in COOrdinate format>

Zur Ausgabe in der normalen Matrix-Darstellung müssen wir die Sparse Matrix in ein reguläres (dicht besetztes) NumPy-Array umwandeln. In realen Anwendungen soll eine solche Konvertierung natürlich unbedingt vermieden werden und würde oft den verfügbaren Arbeitsspeicher sprengen.

In [8]:
print(M.toarray())

[[1. 0. 2. 0. 0.]
 [1. 0. 1. 2. 0.]
 [0. 1. 1. 0. 1.]]


Intern sind nur die von Null verschiedenen Werte als Tripel (_Zeile_, _Spalte_, _Wert_) abgespeichert. Diese Darstellung wird von `print()` ausgegeben.

In [9]:
print(M)

  (0, 0)	1.0
  (0, 2)	2.0
  (1, 0)	1.0
  (1, 2)	1.0
  (1, 3)	2.0
  (2, 1)	1.0
  (2, 2)	1.0
  (2, 4)	1.0


Für die weitere Verarbeitung wird die Sparse Matrix meist in einem noch kompakteren Format gespeichert (**csr** = _row-compressed_ oder **csc** = _column-compressed_):

In [10]:
M1 = M.tocsr()
M1

<3x5 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

`print()` gibt auch in diesem Fall die Tripel-Darstellung aus. Können Sie aus den drei internen Datenfeldern ableiten, wie das CSC-Format aufgebaut ist?

In [11]:
[M1.indptr, M1.indices, M1.data]

[array([0, 2, 5, 8], dtype=int32),
 array([0, 2, 0, 2, 3, 1, 2, 4], dtype=int32),
 array([1., 2., 1., 1., 2., 1., 1., 1.])]

## Bag-of-words mit Scikit-Learn erstellen

In realen Anwendungen erstellen wir den Bag-of-words natürlich nicht selbst, sondern verwenden den optimierten (und recht flexiblen) `CountVectorizer` von **Scikit-Learn**.  Da unsere Texte (d.h. die Gedichtzeilen) bereits als Listen von Token abgespeichert, müssen wir den eingebauten Tokenizer überspringen; stattdessen gibt unsere Identitätsfunktion einfach die vortokenisierte Zeile zurück.

**NB:** Da unsere „Texte“ in `corpus` keine Zeichenketten sondern Listen von Token sind, müssen wir auch `lowercase=False` angeben (was passiert sonst?). Alternativ könnten wir auch alle Vorverarbeitungsschritte mit `analyzer=lambda x: x` überspringen. Dann hätten wir aber keine Möglichkeit mehr, auch Bigramme und Trigramme als Merkmale automatisch erstellen zu lassen.

In [12]:
bow = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
X_bow = bow.fit_transform(corpus)

Der `CountVectorizer` erstellt eine Sparse Matrix im CSR-Format, das für die Repräsentation von Merkmalsvektoren günstig ist (warum?).

In [13]:
X_bow

<14x17 sparse matrix of type '<class 'numpy.int64'>'
	with 38 stored elements in Compressed Sparse Row format>

In [14]:
print(X_bow.toarray())

[[0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1]
 [2 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 2 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 2 0 0 1 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]]


Die _features names_ des BOW zeigen uns, welche Spalte welchem Wort-Typ entspricht. Sie sind _nicht_ in der Merkmalsmatrix selbst mit abgespeichert!

In [15]:
print(bow.get_feature_names_out())

['fort' 'hofft' 'holt' 'hopst' 'horcht' 'klopft' 'koks' 'komm' 'kommt'
 'kotzt' 'mops' 'obst' 'ogottogott' 'otto' 'ottos' 'soso' 'trotzt']


## Weitere lexikalische Merkmale

Neben Bag-of-words-Repräsentationen sind oft auch andere lexikalische Merkmale von Interesse, z.B. Präfix- und Suffix-Zeichenketten beim POS-Tagging. Für die Repräsentation der Gedichtzeilen könnten v.a. Suffix-Zeichenketten relevant sein, da sie auf Reime hindeuten.

Da wir für jedes Objekt nur genau einen Suffix einer bestimmten Länge haben können, handelt es sich nicht um einen Bag-of-words, sondern um ein **One-hot-encoding**. Dieses lässt sich in Scikit-Learn mit dem `DictVectorizer` erstellen, der One-hot-encodings für beliebig viele Merkmale gleichzeitig erzeugt. (**NB:** Lesen Sie immer auch die [zugehörige Dokumentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html), um sich mit den Feinheiten der Scikit-Learn-Funktionen vertraut zu machen!)

In [16]:
def get_suffix(line):
    s = " ".join(line)
    return {'suff1': s[-1:], 'suff2': s[-2:]}
[get_suffix(x) for x in corpus]

[{'suff1': 't', 'suff2': 'zt'},
 {'suff1': 't', 'suff2': 'rt'},
 {'suff1': 't', 'suff2': 'rt'},
 {'suff1': 'o', 'suff2': 'so'},
 {'suff1': 's', 'suff2': 'ks'},
 {'suff1': 't', 'suff2': 'st'},
 {'suff1': 't', 'suff2': 'ht'},
 {'suff1': 's', 'suff2': 'ps'},
 {'suff1': 't', 'suff2': 'ft'},
 {'suff1': 't', 'suff2': 'ft'},
 {'suff1': 'm', 'suff2': 'mm'},
 {'suff1': 't', 'suff2': 'mt'},
 {'suff1': 't', 'suff2': 'zt'},
 {'suff1': 't', 'suff2': 'tt'}]

In [17]:
suff = DictVectorizer(dtype=np.int64) # analog zum BOW-Format
X_suff = suff.fit_transform(get_suffix(x) for x in corpus)
X_suff

<14x15 sparse matrix of type '<class 'numpy.int64'>'
	with 28 stored elements in Compressed Sparse Row format>

In [18]:
print(X_suff.toarray())

[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0]]


In [19]:
print(suff.get_feature_names_out())

['suff1=m' 'suff1=o' 'suff1=s' 'suff1=t' 'suff2=ft' 'suff2=ht' 'suff2=ks'
 'suff2=mm' 'suff2=mt' 'suff2=ps' 'suff2=rt' 'suff2=so' 'suff2=st'
 'suff2=tt' 'suff2=zt']


## Kombination mit numerischen Merkmalen

In vielen Fällen interessieren uns neben dem Bag-of-words und anderen dünn besetzten Merkmalen auch numerische Eigenschaften wie etwas textstatistische Maße. Als Beispiele berechnen wir hier für jede Gedichtzeile ihre **Länge** (= Anzahl Token) und die **durchschnittliche Wortlänge** in der Zeile.

In [20]:
n_tok = [len(x) for x in corpus]
avg_char = lambda line: sum(len(x) for x in line) / len(line)
n_char = [avg_char(x) for x in corpus]

Wir fügen diese Werte in eine numerische, dicht besetzte Merkmalsmatrix als NumPy-Array zusammen (welchen Trick verwenden wir, um das in einer einzigen Codezeile zu erreichen?).

In [21]:
X_stats = np.vstack([n_tok, n_char]).T
print(X_stats.round(2))

[[3.   5.  ]
 [4.   4.  ]
 [4.   4.5 ]
 [2.   4.  ]
 [3.   4.  ]
 [3.   4.  ]
 [2.   5.  ]
 [3.   4.  ]
 [2.   4.5 ]
 [3.   5.  ]
 [4.   4.  ]
 [3.   4.67]
 [3.   4.67]
 [2.   7.  ]]


Für die Anwendung maschineller Lernverfahren müssen wir nun noch beide Gruppen von Merkmalen in eine gemeinsame Matrix kombinieren. Das geht grundsätzlich leicht mit `hstack()`, aber dünn und dicht besetze Matrizen können dabei nicht gemischt werden. Es wäre auch kontraproduktiv, dadurch `X_bow` in eine dicht besetzte Matrix umzuwandeln.

Stattdessen benötigen wir die Sparse-Matrix-Version von `hstack()` aus dem SciPy-Paket:

In [22]:
X = sp.sparse.hstack([X_bow, X_stats])
X

<14x19 sparse matrix of type '<class 'numpy.float64'>'
	with 66 stored elements in COOrdinate format>

Beachten Sie, dass `X_bow` automatisch von einem ganzzahligen Format in eine Gleitkommaformat umgewandelt wurde, um mit den Gleitkommawerten in `X_stats` kompatibel zu sein. Wir überprüfen kurz, dass die Merkmale korrekt zusammengeführt wurden (die Rundung dient zur übersichtlichen Darstellung).

In [23]:
print(X.toarray().round())

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 3. 5.]
 [2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 4. 4.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 4. 4.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 2. 4.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 3. 4.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 3. 4.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 2. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 1. 0. 0. 0. 3. 4.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 2. 4.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 3. 5.]
 [0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 1. 0. 0. 1. 0. 0. 0. 4. 4.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 3. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 3. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 2. 7.]]


Auf die gleiche Weise können wir alle drei Merkmalsgruppen (BOW der Wortformen, One-hot-encoding der Zeilensuffixe und numerische Merkmale) in eine vollständige Merkmalsmatrix kombinieren.

In [24]:
X = sp.sparse.hstack([X_bow, X_suff, X_stats])
X

<14x34 sparse matrix of type '<class 'numpy.float64'>'
	with 94 stored elements in COOrdinate format>

Wir führen diese Schritte in der folgenden **Übung** mit einem wesentlich größeren Datensatz für eine realistische Anwendung durch.